In [1]:
import numpy as np
from qiskit.opflow.primitive_ops import PauliSumOp
from numpy.linalg import inv
from divisiveclustering.datautils import DataUtils
from divisiveclustering.coresetsUtils import Coreset, normalize_np 
import cudaq

/tmp/ipykernel_91250/2368317224.py:2: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow.primitive_ops import PauliSumOp


In [2]:
# From Miriam's code
def get_weighted_mean(coreset, weights):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    mu = np.zeros(dim)
    for i in range(size_coreset):
        mu += coreset[i]*weights[i]
    return mu/sum(weights)

# Helper functions for Hamiltonian creation
def get_weighted_scatter_matrix(coreset, weights):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    T = np.zeros((dim,dim))
    mu = get_weighted_mean(coreset, weights)
    for i in range(size_coreset):
        T += weights[i]*np.outer((coreset[i] - mu),(coreset[i] - mu))
    return T


def Z_i(i, length):
    """ 
    if index i is in the range 0, ..., length-1, the function returns the operator Z_i
    else: the funtion returns the pauli string consisting of pauli I's only
    length is the number of pauli operators tensorised
    """
    pauli_string = ""
    for j in range(length):
        if i == j:
            pauli_string += "Z"
        else:
            pauli_string += "I"
    return pauli_string

def Z_ij(i, j, length):
    pauli_string = ""
    if i == j:
        pauli_string = Z_i(-1, length) # return 'II...II'
    else:
        for k in range(length):
            if k == i or k == j:
                pauli_string += "Z"
            else:
                pauli_string += "I"
    return pauli_string
    
# Create Hamiltonian for our problem
def create_hamiltonian(coreset, weights):

    paulis = []
    pauli_weights = []
    
    T_inv = inv(get_weighted_scatter_matrix(coreset, weights))

    W = sum(weights)

    for i in range(len(coreset)):
        paulis += [Z_i(-1, len(coreset))]
        pauli_weights += [weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))]
    
        for l in range(len(coreset)):
            paulis += [Z_ij(i,l,len(coreset))]
            pauli_weights += [-2*weights[l]*weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))/W]
            
    for j in range(len(coreset)):
        for i in range(j):
            paulis += [Z_ij(i,j,len(coreset))]
            pauli_weights += [2*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))]
            for l in range(len(coreset)):
                paulis += [Z_ij(i,l,len(coreset))]
                pauli_weights += [-2*weights[l]*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))/W]
                paulis += [Z_ij(j,l,len(coreset))]
                pauli_weights += [-2*weights[l]*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))/W]
            
            
    pauli_op = [([pauli,weight]) for pauli,weight in zip(paulis,pauli_weights)]
    hamiltonian = PauliSumOp.from_list([ op for op in pauli_op])
    # we consider the negative of the hamiltonian since VQE approximates the minimum (and not the maximum)
    hamiltonian = -hamiltonian
    
    # Only change made    
    return hamiltonian, paulis, pauli_weights

In [3]:
# Creating 
number_of_qubits = 5
circuit_depth = 1
parameter_count = 4 * circuit_depth * number_of_qubits 

data_utils = DataUtils('../../data')

try:
    raw_data = data_utils.load_dataset()
except:
    raw_data = data_utils.create_dataset(n_samples = 1000)

coresets = Coreset()

coreset_vectors, coreset_weights = coresets.get_coresets(
    data_vectors=raw_data, number_of_runs=10, coreset_numbers=5, size_vec_list=10
)

best_coreset_vectors, best_coreset_weights = coresets.get_best_coresets(raw_data, coreset_vectors, coreset_weights)
normalized_cv = normalize_np(best_coreset_vectors, centralize=True)
normalized_cw = normalize_np(best_coreset_weights, centralize=False)

Data loaded from ../../data/dataset.pickle


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [4]:
hamiltonian_qiskit, paulis, pauli_weights = create_hamiltonian(normalized_cv, normalized_cw)

In [5]:
hamiltonian_qiskit_reduced = hamiltonian_qiskit.reduce()
hamiltonian_qiskit_reduced

PauliSumOp(SparsePauliOp(['IIIII', 'ZZIII', 'ZIZII', 'ZIIZI', 'ZIIIZ', 'IZZII', 'IZIZI', 'IZIIZ', 'IIZZI', 'IIZIZ', 'IIIZZ'],
              coeffs=[-1.23055996+0.j, -0.22409758+0.j,  0.02683374+0.j,  0.14470258+0.j,
  0.27230605+0.j,  0.28018389+0.j,  0.39015886+0.j,  0.20771837+0.j,
 -0.29947509+0.j,  0.7593609 +0.j,  0.29525535+0.j]), coeff=1.0)

In [6]:
len(hamiltonian_qiskit_reduced)

11

In [7]:
operator_dict = {
    "I" : cudaq.spin.i,
    "Z": cudaq.spin.z,
    "X": cudaq.spin.x,
    "Y": cudaq.spin.y
}


hamil_all = cudaq.SpinOperator()
hamil_list = []
for position, pauli in enumerate(paulis):
    hamil_cudaq = cudaq.SpinOperator()
    for i, term in enumerate(pauli):
        hamil_cudaq = hamil_cudaq *operator_dict.get(term)(i)
    hamil_cudaq = hamil_cudaq*pauli_weights[position]
    hamil_list.append(hamil_cudaq)

    
hamil = cudaq.SpinOperator()
for hamil_from_list in hamil_list:
    hamil += hamil_from_list
    
# remove the extra term added because of cudaq.SpinOperator()
hamiltonian_cudaq = hamil - cudaq.spin.i(0)
hamiltonian_cudaq = hamiltonian_cudaq*-1


In [8]:
hamiltonian_cudaq.to_string()

'(-1.23056,-0) I0I1I2I3I4 + (-0.224098,-0) Z0Z1I2I3I4 + (0.0268337,-0) Z0I1Z2I3I4 + (0.144703,-0) Z0I1I2Z3I4 + (0.272306,-0) Z0I1I2I3Z4 + (0.280184,-0) I0Z1Z2I3I4 + (0.390159,-0) I0Z1I2Z3I4 + (0.207718,-0) I0Z1I2I3Z4 + (-0.299475,-0) I0I1Z2Z3I4 + (0.759361,-0) I0I1Z2I3Z4 + (0.295255,-0) I0I1I2Z3Z4'

In [9]:
hamiltonian_qiskit_reduced

PauliSumOp(SparsePauliOp(['IIIII', 'ZZIII', 'ZIZII', 'ZIIZI', 'ZIIIZ', 'IZZII', 'IZIZI', 'IZIIZ', 'IIZZI', 'IIZIZ', 'IIIZZ'],
              coeffs=[-1.23055996+0.j, -0.22409758+0.j,  0.02683374+0.j,  0.14470258+0.j,
  0.27230605+0.j,  0.28018389+0.j,  0.39015886+0.j,  0.20771837+0.j,
 -0.29947509+0.j,  0.7593609 +0.j,  0.29525535+0.j]), coeff=1.0)

In [11]:
hamiltonian_cudaq_np = np.asarray(hamiltonian_cudaq.get_coefficients())

In [14]:
np.equal(hamiltonian_qiskit_reduced.coeffs, hamiltonian_cudaq_np)

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [13]:
hamiltonian_qiskit_reduced.coeffs

array([-1.23055996+0.j, -0.22409758+0.j,  0.02683374+0.j,  0.14470258+0.j,
        0.27230605+0.j,  0.28018389+0.j,  0.39015886+0.j,  0.20771837+0.j,
       -0.29947509+0.j,  0.7593609 +0.j,  0.29525535+0.j])

In [15]:
hamiltonian_cudaq_np

array([-1.23055996-0.j, -0.22409758-0.j,  0.02683374-0.j,  0.14470258-0.j,
        0.27230605-0.j,  0.28018389-0.j,  0.39015886-0.j,  0.20771837-0.j,
       -0.29947509-0.j,  0.7593609 -0.j,  0.29525535-0.j])

In [16]:
hamiltonian_cudaq_np[0]

(-1.2305599617309166-0j)

In [17]:
hamiltonian_qiskit_reduced.coeffs[0]

(-1.230559961730917+0j)